In [ ]:
LMM 파인튜닝
1. 작은 데이터셋 학습 시 과적합 가능성 높음
2. catastrophic forgetting 재앙적 망각 현상
3. 학습 비용이 크다.
4. 데이터 품질이 매우 중요하다. garbage in, garbage out

허깅페이스 로그인
1. 토큰 발급
2. huggingface-cli login 또는 hf auth login << 터미널에 입력하여 허깅페이스 로그인
3. 
from huggingface_hub import login
login()
으로도 가능

토큰 직접입력도 가능
pipe = pipeline("text-generation", model="google/gemma-2b-it", token="YOUR_TOKEN")

gemma 파인튜닝
1. 런팟 48기가 gpu, 파이토치 2.1 디스크 둘다 200gb

In [ ]:
# BLEU
- BLEU (Bilingual Evaluation Understudy)
- 주로 기계 번역(Machine Translation)의 성능을 평가하기 위해 사용되는 지표
- 생성된 텍스트(Candidate)가 정답 텍스트(Reference)와 얼마나 유사한지, 특히 **얼마나 많은 n-gram이 겹치는지**를 측정합니다.


# 구현
- **N-gram 정밀도(Precision) 기반:** BLEU는 **정밀도(Precision)** 를 중심으로 평가
- 즉, 생성된 텍스트에 있는 n-gram이 정답 텍스트에 얼마나 많이 존재하는지를 측정
- **수정된 N-gram 정밀도(Modified N-gram Precision):** 단순히 n-gram이 존재하는지 여부만 보는 것이 아니라, **정답 텍스트에서 n-gram이 나타나는 횟수 이상으로 중복 계산되지 않도록 제한**합니다. 이를 통해 생성된 텍스트가 특정 구절을 반복하여 BLEU 점수를 인위적으로 높이는 것을 방지합니다.
- **길이 패널티(Brevity Penalty):** 생성된 텍스트가 정답 텍스트보다 **지나치게 짧을 경우 점수에 패널티를 부여**합니다. 이를 통해 생성된 텍스트가 너무 짧아서 정보를 충분히 담고 있지 않을 때 점수가 과도하게 높아지는 것을 방지합니다.
- **일반적으로 1~4 gram을 사용**합니다.


In [ ]:
# 런팟 실습 내용
requirements.txt

transformers 
datasets 
evaluate 
trl 
huggingface_hub 
accelerate 
wandb 
scikit-learn


pip install -r requirements.txt 

pip install torch torchvision torchaudio -U 

pip uninstall trl 

pip install trl==0.19.1



from huggingface_hub import login
login()


import torch
import wandb

from sklearn.model_selection import train_test_split

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline, 
    Trainer
)
from transformers.integrations import WandbCallback
from trl import DataCollatorForCompletionOnlyLM
import evaluate

model_name = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_name,
    use_cache=False,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    low_cpu_mem_usage=True,
    attn_implementation="eager",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

import datasets
dataset = datasets.load_dataset("jaehy12/news3")

element = dataset['train'][1]

def get_chat_format(example):
    return [
            {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약 및 관련 키워를 추출해주세요:\n{example['original']}"},
            {"role": "assistant", "content": f"한국어 요약:\n{example['summary']}"}
        ]

formatted = tokenizer.apply_chat_template(
    get_chat_format(element), tokenize=False
)


input_text = """다음 텍스트를 한국어로 간단히 요약해주세요:\n부산의 한 왕복 2차선 도로에서 역주행 사고로 배달 오토바이 운전자인 고등학생이 숨지는 사고가 발생했다.
유족은 '가해자가 사고 후 곧바로 신고하지 않고 늑장 대응해 피해를 키웠다'고 주장하고 있다.
11일 부산진경찰서는 교통사고처리특례법(교통사고처리법)상 업무상 과실치사 혐의로 지난 3일 A(59)씨를 검찰에 불구속 송치했다고 밝혔다.
A씨는 교통사고처리법상 12대 중과실에 해당되는 '중앙선 침범'으로 역주행 교통사고를 일으킨 혐의를 받는다.
경찰에 따르면 스포츠유틸리티차량(SUV) 운전자 A씨는 5월 19일 밤 11시 50분쯤 부산진구 가야고가교 밑 도로에서 중앙선을 넘어 역주행으로 140m를 달려
반대편 차선의 오토바이 운전자 조모(16)군을 들이받았다. 조군은 원동기장치자전거 면허를 취득한 상태였고 헬멧도 쓰고 있었지만 크게 다쳤다.
사고 당일 수술을 받았으나 얼마 후 2차 뇌출혈로 뇌사 판정이 내려졌고, 사고 발생 약 한 달 만인 지난달 16일 끝내 사망했다.
사고를 낸 A씨는 술을 마시거나 약물을 복용한 상태에서 운전하지는 않은 것으로 조사됐다.
경찰 관계자는 'A씨가 자신이 정주행을 하고 오토바이가 역주행을 한 것으로 착각했다고 진술했다'고 설명했다."""


#키워드 추출만 시켰을 때 
def change_inference_chat_format(input_text):
    return [
    {"role": "user", "content": f"{input_text}"},
	{"role": "assistant", "content": """부산의 한 왕복 2차선 도로에서 역주행 사고로 배달 오토바이 운전자인 고등학생이 숨지는 사고가 발생했다.
     유족은 '가해자가 사고 후 곧바로 신고하지 않고 늑장 대응해 피해를 키웠다'고 주장하고 있다."""},
    {"role": "user", "content": "중요한 키워드 5개를 뽑아주세요."},
    {"role": "assistant", "content": ""}
    ]
prompt = change_inference_chat_format(input_text)


inputs = tokenizer.apply_chat_template(
    prompt, 
    tokenize=True, 
    add_generation_prompt=True, 
    return_tensors="pt"
    ).to(model.device)
outputs = model.generate(
    input_ids=inputs.to(model.device), 
    max_new_tokens=256
    )
print(tokenizer.decode(
    outputs[0], 
    skip_special_tokens=True
    ))


#한국어 요약만 시켰을 때
def change_inference_chat_format(input_text):
    return [
    {"role": "user", "content": f"{input_text}"},
    {"role": "assistant", "content": "한국어 요약:\n"}
    ]
prompt = change_inference_chat_format(input_text)


inputs = tokenizer.apply_chat_template(prompt,
                                       tokenize=True, 
                                       add_generation_prompt=True,
                                       return_tensors="pt").to(model.device)

outputs = model.generate(inputs, max_new_tokens=256, use_cache=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


#2가지를 동시에 요청하는 코드 
def change_inference_chat_format(input_text):
    return [
    {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약하고, 관련된 5개의 키워를 추출해주세요:\n{input_text}"},
	{"role": "assistant", "content": ""},
    ]

prompt = change_inference_chat_format(input_text)
inputs = tokenizer.apply_chat_template(prompt, 
                                       tokenize=True, 
                                       add_generation_prompt=True, 
                                       return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=256, use_cache=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))







pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")


def key_word_prompt(input_text, summary_text):
    return [
    {"role": "user", "content": f"{input_text}"},
    {"role": "assistant", "content": f"{summary_text}"},
    {"role": "user", "content": "중요한 키워드 5개를 뽑아주세요."},
    {"role": "assistant", "content": ""}
    ]

def extract_keywords_batch(batch):
    prompts = [key_word_prompt(original, summary) for original, summary in zip(batch["original"], batch["summary"])]

    generated_texts = pipe(prompts, max_new_tokens=150, return_full_text=False)
    keywords = [gen_text[0]["generated_text"] for gen_text in generated_texts]
    batch["keywords"] = keywords
    return batch


sample_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
sample_dataset = sample_dataset.map(extract_keywords_batch, batched=True, batch_size=20)   




def chat_keyword_summary_format(example):
    return [
        {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약 및 관련 키워를 추출해주세요:\n{example['original']}"},
        {"role": "assistant", "content": f"한국어 요약:{example['summary']}\n키워드:{example['keywords']}"}
    ]

formatted = tokenizer.apply_chat_template(
    chat_keyword_summary_format(sample_dataset[0]), tokenize=False
)



EOS_TOKEN = tokenizer.eos_token
def tokenize(element):
    formatted = tokenizer.apply_chat_template(
        chat_keyword_summary_format(element), tokenize=False
    ) + EOS_TOKEN
    outputs = tokenizer(formatted)
    return {
        "input_ids": outputs["input_ids"],
        "attention_mask": outputs["attention_mask"],
    }

tokenized_sample_dataset = sample_dataset.map(tokenize)

tokenized_sample_dataset = tokenized_sample_dataset.train_test_split(
    test_size=0.1, 
    seed=42
)

response_template_ids = tokenizer.encode(
    "<start_of_turn>model\n", 
    add_special_tokens=False
    )

collator = DataCollatorForCompletionOnlyLM(
    response_template_ids, tokenizer=tokenizer
)

wandb.init(project="gemma-2B-it-Full-Fine-Tuning", entity="Your_ID")
training_args = TrainingArguments(
    output_dir="./keywords_gemma_results",
    # num_train_epochs=1, # 1epoch에 250step정도 진행함 
    max_steps=800,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=0,
    weight_decay=0.01,
    learning_rate=2e-4,
    logging_dir="./logs",
    logging_steps=100,
    report_to="wandb",
    )


bleu = evaluate.load("bleu")
acc = evaluate.load("accuracy")


def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        # 모델과 설정에 따라 logits에는 추가적인 텐서들이 포함될 수 있습니다.
        # 예를 들어, past_key_values 같은 것들이 있을 수 있지만,
        # logits는 항상 첫 번째 요소입니다.
        logits = logits[0]
    # 토큰 ID를 얻기 위해 argmax를 수행합니다.
    return logits.argmax(dim=-1)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds는 labels와 같은 형태를 갖습니다.
    # preprocess_logits_for_metrics에서 argmax(-1)가 계산된 후입니다.
    # 하지만 우리는 labels를 한 칸 이동시켜야 합니다.
    labels = labels[:, 1:]
    preds = preds[:, :-1]

    # -100은 DataCollatorForCompletionOnlyLM에서 사용되는 
    # ignore_index의 기본값입니다.
    mask = labels == -100
    # -100을 토크나이저가 디코드할 수 있는 값으로 대체합니다.
    labels[mask] = tokenizer.pad_token_id
    preds[mask] = tokenizer.pad_token_id

    # BLEU 점수는 텍스트를 입력으로 받기 때문에,
    # 토큰 ID에서 텍스트로 변환해야 합니다.
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    
    # accuracy는 정수 리스트를 입력으로 받습니다.
    # 우리는 -100이 아닌 부분만 평가하고 싶기 때문에,
    # 마스크의 부정(~)을 사용합니다.
    accuracy = acc.compute(predictions=preds[~mask], references=labels[~mask])

    return {**bleu_score, **accuracy}


trainer = Trainer(
    args=training_args,
    model=model,
    tokenizer=tokenizer,
    data_collator=collator,
    train_dataset=tokenized_sample_dataset["train"],
    eval_dataset=tokenized_sample_dataset["test"],
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    callbacks=[WandbCallback()]
)



trainer.train()
